# Configurations

In [37]:
from getpass import getpass

token = getpass("Token for BIRD:")

# Ids of learning resources of Serlo which shall be exported (will be
# all entities in production).
entity_ids = [ 1565, 61682, 61688, 40744, 43563, 54749, 177413 ]

Token for BIRD:········


# Helper functions

In [38]:
import mysql.connector

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo",
    charset="latin1"
)

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

# Fetching metadata from the DB

In [50]:
import json

def get_metadata_objects():
    ids_list = "(" + ",".join(map(str, entity_ids)) + ")"
    rows = query(f"""
        select entity.id, type.name,
            JSON_OBJECTAGG(entity_revision_field.field, entity_revision_field.value),
            entity.date, entity_revision.date, entity.current_revision_id,
            license.url, instance.subdomain
        from entity
        join type on entity.type_id = type.id
        join license on license.id = entity.license_id
        join instance on instance.id = entity.instance_id
        join entity_revision on entity_revision.id = entity.current_revision_id
        join entity_revision_field on entity_revision_field.entity_revision_id = entity_revision.id
        where entity.id in {ids_list}
        group by entity.id
    """)
    
    return [ convert_row_to_metadata(row) for row in rows ]

def convert_row_to_metadata(row):
    [
        entity_id, entity_type, params, date_created,
        date_modified, current_revision, license_url,
        instance
    ] = row
    params = json.loads(params)
    
    return {
        "@context": [
            "https://w3id.org/kim/lrmi-profile/draft/context.jsonld",
            { "@language": instance }
        ],
        "id": getIRI(entity_id),
        "type": ["LearningResource", getLearningResourceType(entity_type)],
        "learningResourceType": getLearningResourceType(entity_type),
        "name": decode(params["title"]) if "title" in params else None,
        "description": decode(params["meta_description"]) if "meta_description" in params else None,
        "dateCreated": date_created.isoformat(),
        "dateModified": date_modified.isoformat(),
        "license": { "id": license_url },
        "version": getIRI(current_revision),
        "isAccessibleForFree": True,
        "isFamilyFriendly": True,
        "publisher": "https://serlo.org/",
        "inLanguage": instance,
        "audience": [
            {
                "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/student",
                "prefLabel": {
                    "en": "student",
                    "de": "Schüler*in"
                },
                "type": "Concept",
                "inScheme": { "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/" }
            }
        ],
    }

def getLearningResourceType(entity_type):
    if entity_type == "article" or entity_type == "course-page":
        return "Article"
    if entity_type == "course":
        return "Course"
    if entity_type == "text-exercise-group" or entity_type == "text-exercise":
        return "Quiz"
    if entity_type == "video":
        return "Video"

def getIRI(entity_id):
    return f"https://serlo.org/{entity_id}"

def decode(text):
    return bytes(text, "latin1").decode("utf8")

for metadata_object in get_metadata_objects():
    entity_iri = metadata_object["id"]
    
    display(Markdown(f"## JSON-LD for {entity_iri}"))
    display(Markdown("```json\n" + json.dumps(metadata_object, indent=2) + "\n```"))

## JSON-LD for https://serlo.org/1565

```json
{
  "@context": [
    "https://w3id.org/kim/lrmi-profile/draft/context.jsonld",
    {
      "@language": "de"
    }
  ],
  "id": "https://serlo.org/1565",
  "type": [
    "LearningResource",
    "Article"
  ],
  "learningResourceType": "Article",
  "name": "Sinus, Kosinus und Tangens",
  "description": "Sinus, Kosinus und Tanges beschreiben die Seitenverh\u00e4ltnisse in einem rechtwinkligen Dreieck. Hier kannst du lernen wie du Winkel berechnest, sie sind die wichtigsten trigonometrischen Funktionen. Lerne die Begriffe Hypotenuse, Gegenkathete und Ankathete kennen!",
  "dateCreated": "2014-03-01T20:38:24",
  "dateModified": "2021-09-20T14:21:51",
  "license": {
    "id": "https://creativecommons.org/licenses/by-sa/4.0/deed.de"
  },
  "version": "https://serlo.org/225013",
  "isAccessibleForFree": true,
  "isFamilyFriendly": true,
  "publisher": "https://serlo.org/",
  "inLanguage": "de",
  "audience": [
    {
      "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/student",
      "prefLabel": {
        "en": "student",
        "de": "Sch\u00fcler*in"
      },
      "type": "Concept",
      "inScheme": {
        "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/"
      }
    }
  ]
}
```

## JSON-LD for https://serlo.org/40744

```json
{
  "@context": [
    "https://w3id.org/kim/lrmi-profile/draft/context.jsonld",
    {
      "@language": "de"
    }
  ],
  "id": "https://serlo.org/40744",
  "type": [
    "LearningResource",
    "Video"
  ],
  "learningResourceType": "Video",
  "name": "Winkel konstruieren",
  "description": null,
  "dateCreated": "2015-06-02T17:02:08",
  "dateModified": "2020-01-17T13:30:07",
  "license": {
    "id": "https://creativecommons.org/licenses/by-sa/4.0/deed.de"
  },
  "version": "https://serlo.org/149341",
  "isAccessibleForFree": true,
  "isFamilyFriendly": true,
  "publisher": "https://serlo.org/",
  "inLanguage": "de",
  "audience": [
    {
      "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/student",
      "prefLabel": {
        "en": "student",
        "de": "Sch\u00fcler*in"
      },
      "type": "Concept",
      "inScheme": {
        "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/"
      }
    }
  ]
}
```

## JSON-LD for https://serlo.org/43563

...

```json
{
  "@context": [
    "https://w3id.org/kim/lrmi-profile/draft/context.jsonld",
    {
      "@language": "de"
    }
  ],
  "id": "https://serlo.org/61682",
  "type": [
    "LearningResource",
    "Course"
  ],
  "learningResourceType": "Course",
  "name": "Ganze Zahlen",
  "description": "In diesem Kurs lernst du die ganzen Zahlen kennen.",
  "dateCreated": "2016-08-30T10:22:22",
  "dateModified": "2020-01-17T10:37:01",
  "license": {
    "id": "https://creativecommons.org/licenses/by-sa/4.0/deed.de"
  },
  "version": "https://serlo.org/149202",
  "isAccessibleForFree": true,
  "isFamilyFriendly": true,
  "publisher": "https://serlo.org/",
  "inLanguage": "de",
  "audience": [
    {
      "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/student",
      "prefLabel": {
        "en": "student",
        "de": "Sch\u00fcler*in"
      },
      "type": "Concept",
      "inScheme": {
        "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/"
      }
    }
  ]
}
```

## JSON-LD for https://serlo.org/61688

```json
{
  "@context": [
    "https://w3id.org/kim/lrmi-profile/draft/context.jsonld",
    {
      "@language": "de"
    }
  ],
  "id": "https://serlo.org/61688",
  "type": [
    "LearningResource",
    "Article"
  ],
  "learningResourceType": "Article",
  "name": "Die ganzen Zahlen und der Zahlenstrahl",
  "description": null,
  "dateCreated": "2016-08-30T11:16:19",
  "dateModified": "2020-01-17T10:37:01",
  "license": {
    "id": "https://creativecommons.org/licenses/by-sa/4.0/deed.de"
  },
  "version": "https://serlo.org/149205",
  "isAccessibleForFree": true,
  "isFamilyFriendly": true,
  "publisher": "https://serlo.org/",
  "inLanguage": "de",
  "audience": [
    {
      "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/student",
      "prefLabel": {
        "en": "student",
        "de": "Sch\u00fcler*in"
      },
      "type": "Concept",
      "inScheme": {
        "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/"
      }
    }
  ]
}
```

## JSON-LD for https://serlo.org/177413

```json
{
  "@context": [
    "https://w3id.org/kim/lrmi-profile/draft/context.jsonld",
    {
      "@language": "de"
    }
  ],
  "id": "https://serlo.org/177413",
  "type": [
    "LearningResource",
    "Quiz"
  ],
  "learningResourceType": "Quiz",
  "name": null,
  "description": null,
  "dateCreated": "2020-09-10T11:17:43",
  "dateModified": "2020-09-14T10:45:36",
  "license": {
    "id": "https://creativecommons.org/licenses/by-sa/4.0/deed.de"
  },
  "version": "https://serlo.org/177929",
  "isAccessibleForFree": true,
  "isFamilyFriendly": true,
  "publisher": "https://serlo.org/",
  "inLanguage": "de",
  "audience": [
    {
      "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/student",
      "prefLabel": {
        "en": "student",
        "de": "Sch\u00fcler*in"
      },
      "type": "Concept",
      "inScheme": {
        "id": "http://purl.org/dcx/lrmi-vocabs/educationalAudienceRole/"
      }
    }
  ]
}
```

# Export metadata to BIRD

In [40]:
def export_to_bird(metadata_object):
    pass

for metadata_object in get_metadata_objects():
    export_to_bird(metadata_object)